# Variant Extraction from Hail MatrixTable (All of Us Data)
This notebook extracts a single variant from a Hail MatrixTable using genomic coordinates and exports it for downstream PheWAS analysis.

## Import Libraries and Initialize Hail

In [ ]:
import pandas as pd
import hail as hl
import os

In [ ]:
# Set display options
pd.set_option('display.max_columns', None)

In [ ]:
# Initialize Hail with GRCh38 reference
hl.init(default_reference='GRCh38', idempotent=True)

## Set Workspace Paths and Load MatrixTable
This uses the All of Us hail matrix table for illustration

In [ ]:
# Set bucket environment
bucket = os.getenv('WORKSPACE_BUCKET')

# Path to the All of Us WGS MatrixTable
wgs_path = "gs://fc-aou-datasets-controlled/v8/wgs/short_read/snpindel/exome/splitMT/hail.mt"
wgs = hl.read_matrix_table(wgs_path)


## Define Variant Extraction Function

In [ ]:
def variant_extraction(var_pos, ref_allele, alt_allele, dbsnp_id, output_filename):
    """
    This function extract a single variant from the Hail MatrixTable and export it as TSV format.

    Parameters:
    - var_pos (str): Genomic position, e.g., 'chr17:7579472'
    - ref_allele (str): Reference allele
    - alt_allele (str): Alternate allele
    - dbsnp_id (str): rsID of the variant
    - output_filename (str): Name of the exported TSV file
    """
    
    print(f"Extracting variant at {var_pos} with alleles [{ref_allele}, {alt_allele}]")

    variant_ht = (
        wgs
        .filter_rows(wgs.locus == hl.parse_locus(var_pos))
        .select_rows()
        .entries()
    )
    
    variant_ht1 = (
        variant_ht
        .select(variant_ht.GT, variant_ht.AD)
        .annotate(n_alt=variant_ht.GT.n_alt_alleles())
    )

    variant_ht1 = variant_ht1.filter(variant_ht1.alleles == hl.array([ref_allele, alt_allele]))

    print("Genotype Summary:")

    variant_ht1.group_by(variant_ht1.locus, 
                         variant_ht1.alleles, 
                         variant_ht1.GT).aggregate(n=hl.agg.count()).show(10)

    output_path = f"{bucket}/{output_filename}.tsv"
    
    variant_ht1.filter(hl.is_defined(variant_ht1.GT)).export(output_path)
    
    print(f"Exported to: {output_path}")


## Run Variant Extraction for a Specific Variant

In [ ]:
# Example: Extract TP53 variant rs78378222
variant_extraction(
    var_pos='chr17:7579472',
    ref_allele='C',
    alt_allele='T',
    dbsnp_id='rs78378222',
    output_filename='TP53_variant'
)
